In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# options
pd.options.display.max_columns = None # show all columns

In [10]:
# paths to data
data_dir = os.getcwd() + "/data/"
X_train_path = data_dir + "X_train.csv"
X_test_path = data_dir + "X_test.csv"
y_train_path = data_dir + "y_train.csv"
regis_data_path = data_dir + "vehicle_registration_data_2018.csv"

In [17]:
# Load the data into frames
X_train = pd.read_csv(X_train_path, index_col=0)
y_train = pd.read_csv(y_train_path, index_col=0)
regis_data = pd.read_csv(regis_data_path)

/home/joseph/.virtualenvs/rtw19-hacks/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Expolore X_train

In [12]:
# check out the columns and types in X_train
X_train.dtypes

idx                                      int64
DC_ZIPCODE                               int64
Invoice_Year                             int64
Invoice_Week                             int64
CATEGORY                                object
TIER                                    object
SPEED_RATING_CODE                       object
RIM_DIAMETER_SIZE_CODE                 float64
WIDTH                                  float64
HEIGHT                                 float64
AVG_UNIT_WEIGHT                        float64
Invoice_Month                          float64
SELLING_PRICE                          float64
Monthly_Top_1_Customer_Zip             float64
Monthly_Top_2_Customer_Zip             float64
Monthly_Top_3_Customer_Zip             float64
Monthly_Top_4_Customer_Zip             float64
Monthly_Top_5_Customer_Zip             float64
Monthly_Top_6_Customer_Zip             float64
Monthly_Top_7_Customer_Zip             float64
Monthly_Top_8_Customer_Zip             float64
Monthly_Top_9

In [14]:
# run describe 
X_train.describe()

,idx,DC_ZIPCODE,Invoice_Year,Invoice_Week,RIM_DIAMETER_SIZE_CODE,WIDTH,HEIGHT,AVG_UNIT_WEIGHT,Invoice_Month,SELLING_PRICE,Monthly_Top_1_Customer_Zip,Monthly_Top_2_Customer_Zip,Monthly_Top_3_Customer_Zip,Monthly_Top_4_Customer_Zip,Monthly_Top_5_Customer_Zip,Monthly_Top_6_Customer_Zip,Monthly_Top_7_Customer_Zip,Monthly_Top_8_Customer_Zip,Monthly_Top_9_Customer_Zip,Monthly_Top_10_Customer_Zip,Monthly_Top_1_Customer_Total_Sales,Monthly_Top_2_Customer_Total_Sales,Monthly_Top_3_Customer_Total_Sales,Monthly_Top_4_Customer_Total_Sales,Monthly_Top_5_Customer_Total_Sales,Monthly_Top_6_Customer_Total_Sales,Monthly_Top_7_Customer_Total_Sales,Monthly_Top_8_Customer_Total_Sales,Monthly_Top_9_Customer_Total_Sales,Monthly_Top_10_Customer_Total_Sales
count,2.019036e+06,2.019036e+06,2.019036e+06,2.019036e+06,2.019036e+06,2.019036e+06,2.019036e+06,2.019036e+06,2.019036e+06,2017394.0,2.019036e+06,2.019036e+06,2.019034e+06,2.019034e+06,2.019034e+06,2.019034e+06,2.019032e+06,2.019032e+06,2.019032e+06,2.019032e+06,2.019032e+06,2.019032e+06,2.018971e+06,2.018971e+06,2.018940e+06,2.018940e+06,2.018935e+06,2.018935e+06,2.018935e+06,2.018935e+06
mean,1.009518e+06,5.512789e+04,2.016477e+03,2.677247e+01,1.707623e+01,9.610730e+00,2.859939e+01,3.266373e+01,6.587243e+00,NaN,5.504777e+04,5.499970e+04,5.514500e+04,5.520769e+04,5.528543e+04,5.528743e+04,5.522442e+04,5.532050e+04,5.540878e+04,5.537291e+04,3.690407e+05,2.356325e+05,1.970884e+05,1.684377e+05,1.433929e+05,1.293424e+05,1.195634e+05,1.145104e+05,1.040390e+05,9.682747e+04
std,5.828456e+05,2.771675e+04,4.994850e-01,1.503970e+01,6.234162e+00,1.191372e+00,2.871474e+00,1.447614e+01,3.440257e+00,NaN,2.763403e+04,2.761591e+04,2.771596e+04,2.786329e+04,2.788820e+04,2.787953e+04,2.795052e+04,2.780455e+04,2.780084e+04,2.777710e+04,2.846014e+05,1.486780e+05,1.463554e+05,1.243345e+05,8.319733e+04,6.687201e+04,6.049426e+04,7.312876e+04,5.488834e+04,5.227064e+04
min,0.000000e+00,1.171700e+04,2.016000e+03,1.000000e+00,1.200000e+01,3.240000e+00,1.980000e+01,6.000000e+00,1.000000e+00,-inf,1.110100e+04,6.300000e+01,1.110100e+04,-1.657000e+03,-1.710000e+02,1.247000e+03,-5.635000e+04,-4.849400e+04,-3.855900e+04,-9.556000e+03,-5.902360e+05,-2.166680e+05,-1.578240e+05,-1.556380e+05,-9.389400e+04,-7.182400e+04,-6.570200e+04,-4.780200e+04,-5.029800e+04,-5.306400e+04
25%,5.047588e+05,3.008400e+04,2.016000e+03,1.400000e+01,1.600000e+01,8.800000e+00,2.625000e+01,2.390000e+01,4.000000e+00,74.0,3.003200e+04,3.025300e+04,3.006800e+04,3.003000e+04,3.004000e+04,3.006800e+04,2.990600e+04,2.980300e+04,2.990700e+04,2.981200e+04,1.849960e+05,1.406460e+05,1.193900e+05,1.033600e+05,9.203200e+04,8.530600e+04,7.850200e+04,7.101800e+04,6.697800e+04,6.116800e+04
50%,1.009518e+06,4.837500e+04,2.016000e+03,2.700000e+01,1.700000e+01,9.500000e+00,2.866000e+01,2.983000e+01,7.000000e+00,106.0,4.603800e+04,4.614200e+04,4.620300e+04,4.623100e+04,4.815000e+04,4.817000e+04,4.815000e+04,4.811600e+04,4.844600e+04,4.853200e+04,2.788660e+05,2.020120e+05,1.627900e+05,1.421660e+05,1.290580e+05,1.210540e+05,1.105960e+05,1.024920e+05,9.523400e+04,8.885600e+04
75%,1.514276e+06,8.091600e+04,2.017000e+03,4.000000e+01,1.800000e+01,1.055000e+01,3.110000e+01,3.927000e+01,1.000000e+01,154.0,8.091500e+04,8.090500e+04,8.092100e+04,8.106700e+04,8.092100e+04,8.132100e+04,8.140100e+04,8.106700e+04,8.090600e+04,8.090700e+04,4.460460e+05,2.928280e+05,2.289180e+05,1.974680e+05,1.724060e+05,1.597560e+05,1.498660e+05,1.385860e+05,1.293560e+05,1.216740e+05
max,2.019035e+06,9.800100e+04,2.017000e+03,5.300000e+01,4.600000e+02,2.400000e+01,4.180000e+01,1.000000e+03,1.200000e+01,inf,9.853200e+04,9.853200e+04,9.863200e+04,9.863200e+04,9.863200e+04,9.866500e+04,9.866500e+04,9.863200e+04,9.866500e+04,9.863200e+04,2.554758e+06,1.396878e+06,1.671010e+06,1.927694e+06,8.450160e+05,8.140160e+05,7.150560e+05,1.162378e+06,5.855120e+05,7.855340e+05


In [15]:
# TODO (CLEAN) search for bad/empty/null/missing data
for col in X_train.columns:
    print (col, X_train[col].isna().sum())

idx 0
DC_ZIPCODE 0
Invoice_Year 0
Invoice_Week 0
CATEGORY 0
TIER 0
SPEED_RATING_CODE 0
RIM_DIAMETER_SIZE_CODE 0
WIDTH 0
HEIGHT 0
AVG_UNIT_WEIGHT 0
Invoice_Month 0
SELLING_PRICE 1642
Monthly_Top_1_Customer_Zip 0
Monthly_Top_2_Customer_Zip 0
Monthly_Top_3_Customer_Zip 2
Monthly_Top_4_Customer_Zip 2
Monthly_Top_5_Customer_Zip 2
Monthly_Top_6_Customer_Zip 2
Monthly_Top_7_Customer_Zip 4
Monthly_Top_8_Customer_Zip 4
Monthly_Top_9_Customer_Zip 4
Monthly_Top_10_Customer_Zip 4
Monthly_Top_1_Customer_Total_Sales 4
Monthly_Top_2_Customer_Total_Sales 4
Monthly_Top_3_Customer_Total_Sales 65
Monthly_Top_4_Customer_Total_Sales 65
Monthly_Top_5_Customer_Total_Sales 96
Monthly_Top_6_Customer_Total_Sales 96
Monthly_Top_7_Customer_Total_Sales 101
Monthly_Top_8_Customer_Total_Sales 101
Monthly_Top_9_Customer_Total_Sales 101
Monthly_Top_10_Customer_Total_Sales 101


In [37]:
X_train.groupby(['CATEGORY','TIER','SPEED_RATING_CODE','RIM_DIAMETER_SIZE_CODE']).groups

{('AT Mud Utility',
  'Tier 1',
  'Q',
  17.0): Int64Index([1323781, 1604471], dtype='int64'),
 ('AT Mud Utility',
  'Tier 1',
  'R',
  15.0): Int64Index([  66318,  110460,  110463,  183894,  542678,  553954,  553958,
              553968,  553970,  648596,  648600,  648881,  650882,  650883,
              733110,  733111,  911408,  929962,  929964,  991967, 1106000,
             1129376, 1315537, 1418949, 1612948, 1674540, 1674541, 1712063,
             1806403, 1896182],
            dtype='int64'),
 ('AT Mud Utility',
  'Tier 1',
  'R',
  16.0): Int64Index([ 508577,  690360, 1109954, 1200976, 1567861, 1567884, 1568894,
             1572660, 1574786, 1574794, 2008267],
            dtype='int64'),
 ('AT Mud Utility',
  'Tier 1',
  'R',
  17.0): Int64Index([699429, 1727083, 1727100, 1727127, 1727144], dtype='int64'),
 ('AT Mud Utility',
  'Tier 1',
  'R',
  18.0): Int64Index([565555, 565569], dtype='int64'),
 ('AT Mud Utility', 'Tier 1', 'R', 20.0): Int64Index([1887867], dtype='int64'),

### Explore y_train Data

### Explore Vehicle Registration Data

In [18]:
regis_data.head()

,COUNTOF,YEAR,CatManSegment,CatManCategory,FileYear,Speed_Source,Speed_Target,Speed,Tire_Ply,MAKE,MODEL,TRIM,CATEGORY,SECTION_WIDTH,ASPECT_RATIO,TIRE_DIAMETER,ZIP_CODE,TIRESIZESEARCH,TOTAL_VEHICLE_COUNT,RANK_COL
0,4,2016,Consumer,Light Truck,201810,NaN,NaN,S,E,FORD,F250,SUPER DUTY,DOMESTIC LIGHT TRUCK,245,75.0,17.0,1009,2457517,0.4,1
1,4,2010,Consumer,Performance Sport,201810,H,HP,H,NaN,TOYOTA,COROLLA,BASE/S/LE/XLE,IMPORT CAR,205,55.0,16.0,1009,2055516,1.5,1
2,4,2008,Consumer,Light Truck,201810,NaN,NaN,Q,C,CHEVROLET,SILVERADO,K1500,DOMESTIC LIGHT TRUCK,245,70.0,17.0,1009,2457017,1.0,1
3,4,2008,Consumer,Light Truck,201810,NaN,NaN,S,NaN,CHEVROLET,SILVERADO,K1500,DOMESTIC LIGHT TRUCK,265,70.0,17.0,1009,2657017,3.0,1
4,4,2016,Consumer,Light Truck,201810,NaN,NaN,R,E,FORD,F250,SUPER DUTY,DOMESTIC LIGHT TRUCK,245,75.0,17.0,1009,2457517,0.4,1
